In [1]:
import torch
from tqdm import tqdm
import pandas as pd
from ast import literal_eval
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# bitsandbytes 안 깔려있으면 주석 해제 후 설치
# !pip install bitsandbytes

In [ ]:
model_name_or_path = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
inference_csv = 'data/test.csv'
output_csv = 'zeroshot_result/output_change_system_m.csv'

system_message = """당신은 객관식 문제의 정답을 찾는 데 특화된 AI입니다.
- 지문을 분석해 가장 적합한 정답을 찾아야 합니다.
- 정답은 **숫자(1, 2, 3, 4, 5)** 형식으로만 작성합니다.
- 정답 외의 어떤 설명도 포함하지 마세요."""

PROMPT_NO_QUESTION_PLUS = """
정답은 반드시 숫자(1, 2, 3, 4, 5)로만 적어야합니다. 예시) 정답: 1

지문:
{paragraph}

질문:
{question}

선택지:
{choices}


정답:"""

PROMPT_QUESTION_PLUS = """
정답은 반드시 숫자(1, 2, 3, 4, 5)로만 적어야합니다. 예시) 정답: 1

지문:
{paragraph}

질문:
{question}

<보기>:
{question_plus}

선택지:
{choices}


정답:"""



In [3]:
# torch.bfloat16과 load_in_4bit 중 택 1
# bfloat16이 더 빠르나, OOM이 발생할 경우 load_in_4bit 사용
# 추론 시, 긴 지문+보기가 나올 경우 OOM이 발생할 수 있음
# 둘 중, 하나를 주석 처리해주세요.
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, trust_remote_code=True,
    device_map="auto",
    #torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### ### 이 셀 아래부터는 수정하지 않아도 됩니다

In [5]:
def load_and_process_data(file_path):
    dataset = pd.read_csv(file_path)
    
    records = []
    for _, row in dataset.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', None),
            'explanation': row.get('explanation', None)
        }
        records.append(record)

    df = pd.DataFrame(records)
    df['question_plus'] = df['question_plus'].fillna('')
    return Dataset.from_pandas(df)

def format_inference_dataset(test_df):
    test_dataset = []
    for i, row in test_df.iterrows():
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(row["choices"])])
        len_choices = len(row["choices"])
        
        # <보기>가 있을 때
        if row["question_plus"]:
            user_message = PROMPT_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                question_plus=row["question_plus"],
                choices=choices_string,
            )
        # <보기>가 없을 때
        else:
            user_message = PROMPT_NO_QUESTION_PLUS.format(
                paragraph=row["paragraph"],
                question=row["question"],
                choices=choices_string,
            )

        test_dataset.append(
            {
                "id": row["id"],
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_message},
                ],
                "label": row["answer"],
                "len_choices": len_choices,
            }
        )

    return test_dataset

In [6]:
model.eval()
model.to("cuda")

test_df = load_and_process_data(inference_csv).to_pandas()
test_dataset = format_inference_dataset(test_df)  # test_dataset은 list of dict

infer_results = []

with torch.inference_mode():
    for data in tqdm(test_dataset, total=len(test_dataset)):
        input_tensor = tokenizer.apply_chat_template(
            data["messages"],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        outputs = model.generate(
            input_tensor,
            max_new_tokens=1,
            pad_token_id=tokenizer.eos_token_id,   
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False,
            temperature=None, 
            top_p=None,
            top_k=None,
        )

        response = tokenizer.decode(outputs[0][input_tensor.size(1):], skip_special_tokens=True)

        infer_results.append({
            "id": data["id"],
            "answer": response.strip()
        })

pd.DataFrame(infer_results).to_csv(output_csv, index=False)

You shouldn't move a model that is dispatched using accelerate hooks.
  0%|          | 0/869 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
100%|██████████| 869/869 [07:25<00:00,  1.95it/s]
